In [1]:
import datetime as dt
import logging
import sys

import polars as pl

from backtest_lib.backtest import Backtest
from backtest_lib.examples.get_sp500_historical import get_sp500_market_view
from backtest_lib.market import MarketView
from backtest_lib.market.polars_impl import PolarsPastView, SeriesUniverseMapping
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy import Decision

logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

market_view = get_sp500_market_view(
    start=dt.datetime.fromisoformat("2010-01-03"),
)
# market_view = pkl.load(open("2010-sp500-market-view.pkl", "rb"))
# market_view = market_view.truncated_to(500)


universe = tuple(market_view.prices.close.by_security.as_df().columns)

[*********************100%***********************]  811 of 811 completed

[ERROR] yfinance: 
148 Failed downloads:
[ERROR] yfinance: ['CPGX', 'GENZ', 'NOVL', 'XTO', 'HCBK', 'CVC', 'LVLT', 'WFM', 'TWC', 'STJ', 'RAI', 'CSC', 'MWW', 'LXK', 'BRCM', 'SPLS', 'TSO', 'JOYG', 'JDSU', 'APOL', 'NVLS', 'MJN', 'SNI', 'KFT', 'CFN', 'TYC', 'BJS', 'EK', 'SIAL', 'LUK', 'JOY', 'WYN', 'SWY', 'PGN', 'LLTC', 'WLP', 'LO', 'BXLT', 'NYX', 'ARG', 'FDO', 'MIL', 'KORS', 'CEPH', 'JNS', 'BCR', 'KRFT', 'CMCSK', 'GGP', 'GMCR', 'HSP', 'DPS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2010-01-03 00:00:00 -> 2025-11-11 22:56:22.199027)')
[ERROR] yfinance: ['FRX', 'XLNX', 'APC', 'KSU', 'FLT', 'LM', 'HFC', 'DNR', 'LSI', 'AKS', 'ABMD', 'ADS', 'RE', 'DISCK', 'MON', 'RHT', 'MRO', 'CHK', 'PBCT', 'DFS', 'DISH', 'COG', 'TWTR', 'WPX', 'XL', 'CXO', 'WLTW', 'CCR', 'AGN', 'X', 'SWN', 'SIVB', 'SRCL', 'CELG', 'MXIM', 'LLL', 'VAR', 'NLSN', 'PXD', 'XEC', 'DRE', 'DO', 'DF', 'SAI', 'ATVI', 'FSR', 'CERN', 'MNK', 'VIAB', 'FBHS', 'ESV', 'FRC', 'FLIR', 'JNPR', 'PDCO', 'HES', 'RDC', 'MDP', 

In [2]:
pf = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series([1 / len(universe)] * len(universe))
    ),
)


def get_even_portfolio(full_universe, partial_universe: list[str]) -> WeightedPortfolio:
    zeroed_securities = list(set(full_universe) - set(partial_universe))
    return WeightedPortfolio(
        cash=0,
        holdings=SeriesUniverseMapping.from_names_and_data(
            partial_universe + zeroed_securities,
            pl.Series(
                ([1 / len(partial_universe)] * len(partial_universe))
                + ([0.0] * len(zeroed_securities))
            ),
        ),
    )


def index_strategy(universe, current_portfolio, market, ctx) -> Decision:
    tradable = [x for x, y in market.tradable.by_period[-1].items() if y]
    even_portfolio = get_even_portfolio(universe, tradable)

    return Decision(even_portfolio)


df = market_view.prices.close.by_security.as_df()
cols = df.columns
ewma = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(date=market_view.periods)
)

ewma_past_view = PolarsPastView.from_dataframe(ewma)

market_view = MarketView(
    prices=market_view.prices,
    tradable=market_view.tradable,
    signals={"ewma_ratio": ewma_past_view},
    periods=market_view.periods,
)


def ewma_strategy(universe, current_portfolio, market, ctx):
    tradable_today = market.tradable.by_period[-1]
    latest_ewma_ratio = (
        market.signals["ewma_ratio"]
        .by_security[[sec for sec in universe if tradable_today[sec] is True]]
        .by_period[-1]
    )

    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return Decision(target=WeightedPortfolio(cash=0, holdings=weights))


benchmark_backtest = Backtest(
    universe=universe,
    strategy=index_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

ewma_backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

TypeError: cannot create expression literal for value of type Array1DDTView.

Hint: Pass `allow_object=True` to accept any value and create a literal of type Object.

In [ ]:
print(
    all(
        x == 0
        for x in ewma_backtest.market_view.prices.close.by_security.as_df().row(0)
    )
)
ewma_results = ewma_backtest.run()

In [ ]:
print(f"total return: {(ewma_results.total_growth - 1) * 100:.2f}%")
sorted_weights = sorted(
    ewma_backtest._current_portfolio.holdings.items(), key=lambda x: x[1], reverse=True
)
sorted_weights

In [ ]:
results = benchmark_backtest.run()
print(f"total return: {(results.total_growth - 1) * 100:.2f}%")

# what a strategy

In [ ]:
len(market_view.tradable.by_period)

In [ ]:
import altair as alt

final_allocations = (
    ewma_results.allocation_history.by_period[-1:]
    .by_security.as_df()
    .lazy()
    .unpivot()
    .filter(pl.col("value") > 0)
    .collect()
)
alt.Chart(final_allocations).mark_bar().encode(
    x="variable",
    y=alt.Y(
        "value",
        scale=alt.Scale(
            domain=[
                final_allocations["value"].min() * 0.99,
                final_allocations["value"].max() * 1.01,
            ],
            clamp=True,
        ),
    ),
)

In [ ]:
final_allocations_series = ewma_results.allocation_history.by_period[-1]
final_allocations = pl.DataFrame(
    data={
        "security": final_allocations_series.names,
        "value": final_allocations_series.as_series(),
    }
).filter(pl.col("value") > 0)

alt.Chart(final_allocations).mark_bar().encode(
    x="security",
    y=alt.Y(
        "value",
        scale=alt.Scale(
            domain=[
                final_allocations["value"].min() * 0.99,
                final_allocations["value"].max() * 1.01,
            ],
            clamp=True,
        ),
    ),
)

In [ ]:
nvda_pnl = list(
    results.returns_contribution.by_security["NVDA"]
    .as_series()
    .rolling_mean(220, min_samples=10)
)

nvda_pnl_values = [x for x in nvda_pnl if x is not None]

dates = ewma_backtest.market_view.periods

alt.Chart(pl.DataFrame({"pnl": nvda_pnl, "date": dates})).mark_line().encode(
    x="date",
    y=alt.Y(
        "pnl",
        scale=alt.Scale(
            domain=[min(nvda_pnl_values) * 0.98, max(nvda_pnl_values) * 1.02],
            clamp=True,
        ),
    ),
)

In [ ]:
num_securities = len(ewma_results.returns_contribution.by_security)
num_securities

In [ ]:
results.allocation_history